# Using Sentinel-1 and -2 data to build a land use classification model
## Overview
This notebook will take you through a workflow for processing Sentinel-1 and -2 data.

## Required datafiles
This notebook requires three files:
- A shapefile that contains land use information: `ST_training data_updated_1130points.shp`
- A raster sentinel-1 vv file (single VV polarisation): `vv-0922_0923-full_ST.tif`
- A raster sentinel-1 vh file (single VH polarisation): `vh-0922_0923-full_ST.tif`

[Sentinel-1](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-1-sar/product-overview/polarimetry) can collect several different images from the same series of pulses by using its antenna to receive specific polarisations simultaneously. Sentinel-1 is a phase-preserving dual polarisation SAR system. It can transmit a signal in either horizontal (H) or vertical (V) polarisation, and then receive in both H and V polarisations.

## Downloading Sentinel-1 data

The sentinel-1 data files are available via an s3 bucket. To download the files locally we will use the aws client. First, check the files are available on the s3 bucket:

In [ ]:
! aws s3 ls s3://easi-asia-dc-data-projects/ctu/sentinel-1/

Next, we can download the files into our currenty directory. First, check your current directory:

In [ ]:
! pwd

Now we're ready to download

In [ ]:
! aws s3 cp s3://easi-asia-dc-data/staging/ctu/sentinel-1/vh-0922_0923-full_ST.tif vh-0922_0923-full_ST.tif

In [ ]:
! aws s3 cp s3://easi-asia-dc-data/staging/ctu/sentinel-1/vv-0922_0923-full_ST.tif vv-0922_0923-full_ST.tif

## Setting up your notebook

We will now setup your notebook so that you have all of the required python modules to query, analyse and plot the data.

### Import modules

In [1]:
%%time
# Basic plots
%matplotlib inline
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Common imports and settings
import os, sys
os.environ['USE_PYGEOS'] = '0'
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr

# Datacube
import datacube
from datacube.utils.rio import configure_s3_access
from datacube.utils import masking
from datacube.utils.cog import write_cog
# https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools
from dea_tools.plotting import display_map, rgb
from dea_tools.datahandling import mostcommon_crs

# EASI defaults
easinotebooksrepo = '/home/jovyan/easi-notebooks'
if easinotebooksrepo not in sys.path: sys.path.append(easinotebooksrepo)
from easi_tools import EasiDefaults, xarray_object_size, notebook_utils
# from easi_tools import load_s2l2a_with_offset
from easi_tools.load_s2l2a import load_s2l2a_with_offset

# Data tools
import numpy as np
from datetime import datetime

# Datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-algo/blob/main/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-algo/blob/main/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
from utils import load_data_geo
import rasterio
import os
import rioxarray
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

from deafrica_tools.bandindices import calculate_indices
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import joblib

CPU times: user 8.7 s, sys: 1.53 s, total: 10.2 s
Wall time: 9.91 s


### Setting up dask
We will now set up a dask cluster. The below code sets up a 'dask gateway' which allows processing of up to 10 worker nodes (cores) for fast parallel processing.

The below code usually takes about 3 minutes to run while dask starts up a new cluster. Once the cluser has initiliased, you can click `Launch dashboard in JupyterLab` to monitor the cluster acitivity.

In [2]:
%%time
# Dask gateway
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
client

An existing cluster was found. Connecting to: easihub.1f3e35a8b8d34990958f809dcd1c6705
CPU times: user 27 ms, sys: 4.29 ms, total: 31.3 ms
Wall time: 131 ms


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.asia.easi-eo.solutions/services/dask-gateway/clusters/easihub.1f3e35a8b8d34990958f809dcd1c6705/status,


### Loading the datacube
Next we load the datacube object. This datacube is what we will use to read the satellite data into memory. We must also configure s3 access to the Sentinel-2 data which is available in the cloud.

In [3]:
# Initialise a datacube object
dc = datacube.Datacube()

# Configure s3 access
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

### Defining a search query
We now need to query the sentinel-2 cloud database on EASI. To extract data we define a query dictionary that contains search criteria such as spatial and temporal limits.

In [4]:
# Specify the start and end times 
min_date = '2022-09-01' # Thời gian bắt đầu lấy data cho quá trình train
max_date = '2023-10-01' # Thời gian kết thúc lấy data cho quá trình train
# Just do 1 month for testing
# max_date = '2022-10-01' # Thời gian kết thúc lấy data cho quá trình train

# Specify a spatail region to search using latitude/longitude cooridinates
min_longitude, max_longitude = (105.5, 106.4)
min_latitude, max_latitude = (9.2, 10.0)

# Specify the product. In this case we want to use Sentinel-2 Level-2A data
product = 's2_l2a'

# Construct the search query dictionary
query = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
    'time': (min_date, max_date),           # Any parsable date strings
}

### Most common CRS
We can select the most appropriate CRS projection using the `notebook_utils` helper function. This function takes into account the data product and spatial location and determines the most common CRS for that region. It generally takes 1-2 seconds to find the most common CRS.

In [5]:
# Most common CRS
native_crs = notebook_utils.mostcommon_crs(dc, query)
print(f'Most common native CRS: {native_crs}')

Most common native CRS: EPSG:32648


### Target xarray parameters
Next we can define which target parameters we would like to load into our xarray object. In general we need to:
 - Select a set of measurements to load
 - Define an appropriate output CRS and resolution
 - Decide on how we group the data (usually we `groupby` input scenes on the same day to a single time layer).
 - Select a reasonable Dask chunk size (this should be adjusted depending on the spatial and resolution parameters you choose

In [6]:
# Specify the spectral band measurements we want to use for a classification algorithm
measurements = ['blue', 'green', 'red', 'nir', 'scl']

load_params = {
    'measurements': measurements,                   # Selected measurement or alias names
    'output_crs': native_crs,                       # Target EPSG code
    'resolution': (-10, 10),                        # Target resolution
    'group_by': 'solar_day',                        # Scene grouping
    'dask_chunks': {'x': 2048, 'y': 2048},          # Dask chunks
}

### Loading the data
We are now read to load the data into our datacube object. For sentinel-2 data we can use the `load_s2l2a_with_offset` helper function so that the sentinel-2 scale and offset coefficients are applied correctly. More information on this issue is discussed [here](https://github.com/csiro-easi/easi-notebooks/blob/main/datasets/sentinel-2-l2a.ipynb).

In [7]:
%%time
# The replacement "dc.load()" function for this product
data = load_s2l2a_with_offset(
    dc,
    query | load_params   # Combine the two dicts that contain our search and load parameters
)

# This line prints the total size of the dataset hat was loaded
notebook_utils.heading(notebook_utils.xarray_object_size(data))

display(data)

No datasets require offset correction. Apply scale (no offset) to all layers


<xarray.Dataset>
Dimensions:      (time: 151, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-02T03:35:23.960000 ... 2023-09...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables:
    blue         (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

CPU times: user 1.5 s, sys: 22.8 ms, total: 1.53 s
Wall time: 1.63 s


### Masking



In [8]:
# # Get the scale factor and offset from the measurement metadata
# measurement_info = dc.list_measurements().loc[query1['product']].loc[query1['measurements']]  # Pandas dataframe
# display(measurement_info)

# The "SCL" band contains quality flags and information. The details can also be found in the metadata.
flag_name = 'scl'
flag_desc = masking.describe_variable_flags(data[flag_name])  # Pandas dataframe
display(flag_desc)
display(flag_desc.loc['qa'].values[1])

,bits,values,description
qa,"[0, 1, 2, 3, 4, 5, 6, 7]","{'0': 'no data', '1': 'saturated or defective'...",Sen2Cor Scene Classification


{'0': 'no data',
 '1': 'saturated or defective',
 '2': 'dark area pixels',
 '3': 'cloud shadows',
 '4': 'vegetation',
 '5': 'bare soils',
 '6': 'water',
 '7': 'unclassified',
 '8': 'cloud medium probability',
 '9': 'cloud high probability',
 '10': 'thin cirrus',
 '11': 'snow or ice'}

In [9]:
# Create a "data quality" Mask layer
flags_def = flag_desc.loc['qa'].values[1]
good_pixel_flags = [flags_def[str(i)] for i in [2, 4, 5, 6]]  # To pass strings to enum_to_bool()

# enum_to_bool calculates the pixel-wise "or" of each set of pixels given by good_pixel_flags
# 1 = good data
# 0 = "bad" data

good_pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)  # -> DataArray
# display(good_pixel_mask)  # Type: bool

In [10]:
data_layer_names = [x for x in data.data_vars if x != 'scl']

In [11]:
data_layer_names

['blue', 'green', 'red', 'nir']

In [12]:
from dask.distributed import wait, progress


rs = []
for layer_name in data_layer_names:
    # Apply valid mask (calculated above) and good pixel mask
    # layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask)
    layer = data[[layer_name]].where(good_pixel_mask)
    rs.append(layer)
    
# Calculate intermediate result
result = xr.merge(rs).persist()

/env/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 9.85 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [13]:
# This takes 5min 17.8s
progress(result)

VBox()

In [14]:
ds1 = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = ds1["NDVI"]
average_ndvi = ndvi.resample(time='1M').mean().persist()  ## tính mean cho từng tháng -> time = 12

In [15]:
# This takes 48.8s
progress(average_ndvi)

VBox()

In [16]:
%%time
average_ndvi = average_ndvi.compute()

CPU times: user 4.12 s, sys: 2.61 s, total: 6.72 s
Wall time: 12.8 s


In [ ]:
%%time
plt.imshow(average_ndvi.isel(time=5))

In [17]:
%%time
filled_ds = average_ndvi.bfill(dim='time')
filled_ds = filled_ds.ffill(dim='time')

CPU times: user 2.84 s, sys: 683 ms, total: 3.52 s
Wall time: 3.52 s


In [ ]:
%%time
plt.imshow(filled_ds.isel(time=5))

In [18]:
train_path = "train/ST_training data_updated_1130points.shp"  # đường dẫn shp file train
train = load_data_geo(train_path)
train.head()

,No,X,Y,LU2022,Hientrang,HT_code,geometry
0,1.0,603860.819,1081162.862,Pomelo,CLN,5,POINT (603860.819 1081162.862)
1,2.0,601306.410,1082782.940,Pomelo,CLN,5,POINT (601306.410 1082782.940)
2,3.0,601084.510,1081351.870,Pomelo,CLN,5,POINT (601084.510 1081351.870)
3,4.0,602193.760,1079205.220,Pomelo,CLN,5,POINT (602193.760 1079205.220)
4,5.0,602459.000,1080946.000,Pomelo,CLN,5,POINT (602459.000 1080946.000)


In [19]:
## load vh vv
VH_file = "vh-0922_0923-full_ST.tif" # đường dẫn tif file sen1 VH
VV_file = "vv-0922_0923-full_ST.tif" # đường dẫn tif file sen1 VV
dsvv = rioxarray.open_rasterio(VV_file)
dsvh = rioxarray.open_rasterio(VH_file)

Warning 1: TIFFReadDirectory:Invalid data type for tag StripByteCounts
Warning 1: TIFFFetchNormalTag:ASCII value for tag "GeoASCIIParams" contains null byte in value; value incorrectly truncated during reading due to implementation limitations
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Invalid data type for tag StripByteCounts
Warning 1: TIFFFetchNormalTag:ASCII value for tag "GeoASCIIParams" contains null byte in value; value incorrectly truncated during reading due to implementation limitations
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


In [23]:
%%time

loaded_datasets = {}
for idx, point in train.iterrows():
    key = f"point_{idx + 1}"
    try:
        ndvi_data = filled_ds.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
        vh_data = dsvh.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
        vv_data = dsvv.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
        loaded_datasets[key] = {
            "data": np.concatenate((ndvi_data, vh_data, vv_data)),
            "label": point.HT_code
                               }
    except Exception as e:
        # loaded_datasets[key] = None
        print(e)

0
100
200
300
400
500
600
700
800
900
1000
1100
CPU times: user 3.07 s, sys: 155 ms, total: 3.22 s
Wall time: 3.21 s


In [24]:
label_mapping = {
    "Lua tom": "1",
    "Lua": "2",
    "CHN": "3",
    "CLN": "4",
    "TS": "5",
    "Song": "6",
    "Dat xay dung": "7",
    "Rung": "8"
}
label_encoder = LabelEncoder()

# Fit and transform the labels
labels = train.Hientrang.values
numeric_labels = label_encoder.fit_transform([label_mapping[label] for label in labels])

In [25]:
X = []
x_new = []
lb_new = []
for k, v in loaded_datasets.items():
    X.append(v)
for i in range(len(X)):
    if X[i] is not None:
        x_new.append(X[i]["data"])
        lb_new.append(numeric_labels[i])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(x_new, lb_new, test_size=0.3, random_state=42)

In [27]:
%%time
# Tạo RandomForestClassifier mặc định để sử dụng làm mô hình ban đầu trong pipeline
base_model = RandomForestClassifier(random_state=42, n_jobs=-1)

# Tạo pipeline
pipeline = Pipeline([
    # ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', base_model),
])
# Thiết lập các tham số bạn muốn tối ưu hóa
param_grid = {
    'classifier__n_estimators': [100, 300, 500, 700, 1000],
    'classifier__max_depth': [6, 8, 10, 15, 20],
    'classifier__criterion': ['gini', 'entropy'],
}

# Sử dụng GridSearchCV để tìm bộ tham số tốt nhất
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# In ra bộ tham số tốt nhất
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Dự đoán trên tập kiểm tra
y_pred = grid_search.predict(X_test)

# Đánh giá kết quả
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Best Parameters: {'classifier__criterion': 'gini', 'classifier__max_depth': 15, 'classifier__n_estimators': 500}
Accuracy: 0.8849557522123894
CPU times: user 7.48 s, sys: 837 ms, total: 8.32 s
Wall time: 1min 25s


2024-03-12 10:04:37,912 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [ ]:
dir_save_model = "model_train"
if not os.path.exists(dir_save_model):
    os.mkdir(dir_save_model)
joblib.dump(grid_search, os.path.join(dir_save_model, "model.joblib"))

In [ ]:
client.close()
cluster.close()